In [14]:
import numpy as np

In [15]:
!python --version

Python 3.11.5


In [16]:
#demo for evaluation
#https://gtuckerkellogg.github.io/pairwise/demo/

In [17]:
""" 
F(i, j) = max(
            F(i - 1, j - 1) + s(xi, yj),
            maxk=0…i-1F(k, j) - γ(i - k),
            maxk=0…j-1F(i, k) - γ(j - k),
            0)   

"""

' \nF(i, j) = max(\n            F(i - 1, j - 1) + s(xi, yj),\n            maxk=0…i-1F(k, j) - γ(i - k),\n            maxk=0…j-1F(i, k) - γ(j - k),\n            0)   \n\n'

In [18]:
scores = {
    "match" : 2,
    "mismatch" : -1,
    "gap": 0
}

def s(xi,xj):
    return scores["match"] if xi == xj else scores["mismatch"]

def match_mismatch(i,j, xi, xj, matrix):
    return matrix[i-1,j-1] + s(xi, xj)

#maxk=0…i-1F(k, j) - γ(i - k)
def γ(n):
    return n * scores["gap"]

def row_gaps_score(i,j):
    gap_scores = []
    for k in range((i)):
        gap_score = matrix[k,j] - γ(i-k)
        gap_scores.append(gap_score)
    
    return np.max(gap_scores), np.argmax(gap_scores)

#maxk=0…j-1F(i, k) - γ(j - k),
def col_gaps_score(i,j):
    gap_scores = []
    for k in range((j)):
        gap_score = matrix[i,k] - γ(j-k)
        gap_scores.append(gap_score)
    return np.max(gap_scores), np.argmax(gap_scores)

bottom_sequence = "PAWHEAE"
top_sequence = "HEAGAWGHEE"
n_rows = len(bottom_sequence)+1
n_cols = len(top_sequence)+1

matrix = np.zeros((n_rows, n_cols))
traceback_dic = {}

for i in range(1,n_rows):
    for j in range(1,n_cols):
        #print(f"i: {i}, j: {j}")
        xi = bottom_sequence[i-1]
        xj = top_sequence[j-1]
        score1 = match_mismatch(i,j, xi, xj, matrix)
        score2, kth_row = row_gaps_score(i,j)
        score3, kth_col = col_gaps_score(i,j)
        top_score = max(score1, 
                        score2, 
                        score3, 
                        0)
        
        matrix[i,j] = top_score

        top_idx = tuple([t for t, value in enumerate([score1, score2, score3]) if value == top_score])
        traceback_dic[f"{i},{j}"] = top_idx if top_score > 0 else None

traceback_dic

{'1,1': None,
 '1,2': None,
 '1,3': None,
 '1,4': None,
 '1,5': None,
 '1,6': None,
 '1,7': None,
 '1,8': None,
 '1,9': None,
 '1,10': None,
 '2,1': None,
 '2,2': None,
 '2,3': (0,),
 '2,4': (2,),
 '2,5': (0, 2),
 '2,6': (2,),
 '2,7': (2,),
 '2,8': (2,),
 '2,9': (2,),
 '2,10': (2,),
 '3,1': None,
 '3,2': None,
 '3,3': (1,),
 '3,4': (1, 2),
 '3,5': (1, 2),
 '3,6': (0,),
 '3,7': (2,),
 '3,8': (2,),
 '3,9': (2,),
 '3,10': (2,),
 '4,1': (0,),
 '4,2': (2,),
 '4,3': (1, 2),
 '4,4': (1, 2),
 '4,5': (1, 2),
 '4,6': (1,),
 '4,7': (1, 2),
 '4,8': (0,),
 '4,9': (2,),
 '4,10': (2,),
 '5,1': (1,),
 '5,2': (0,),
 '5,3': (2,),
 '5,4': (2,),
 '5,5': (2,),
 '5,6': (1, 2),
 '5,7': (1, 2),
 '5,8': (1,),
 '5,9': (0,),
 '5,10': (0, 2),
 '6,1': (1,),
 '6,2': (1,),
 '6,3': (0,),
 '6,4': (2,),
 '6,5': (0, 2),
 '6,6': (2,),
 '6,7': (2,),
 '6,8': (1, 2),
 '6,9': (1,),
 '6,10': (1, 2),
 '7,1': (1,),
 '7,2': (0, 1),
 '7,3': (1,),
 '7,4': (1, 2),
 '7,5': (1, 2),
 '7,6': (1, 2),
 '7,7': (1, 2),
 '7,8': (1, 2),
 '7,

In [19]:
matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 0.,  0.,  0.,  2.,  2.,  2.,  4.,  4.,  4.,  4.,  4.],
       [ 0.,  2.,  2.,  2.,  2.,  2.,  4.,  4.,  6.,  6.,  6.],
       [ 0.,  2.,  4.,  4.,  4.,  4.,  4.,  4.,  6.,  8.,  8.],
       [ 0.,  2.,  4.,  6.,  6.,  6.,  6.,  6.,  6.,  8.,  8.],
       [ 0.,  2.,  4.,  6.,  6.,  6.,  6.,  6.,  6.,  8., 10.]])